In [1]:
import numpy as np
import pandas as pd
import random as r
import math
import os
from datetime import timedelta
import statistics as stats 

import matplotlib.pyplot as plot
import matplotlib.ticker as mtick
import networkx as nx
import seaborn as sns

from pyomo.environ import *
from pyomo.opt import SolverFactory
#from gurobipy import GRB

In [2]:
os.chdir(os.getcwd()+'/outputs/strategic_decisions')

In [3]:
#various budgets tested in beggining of forecasting period
budget_levels = list(range(3000000, 5000000+1, 1000000))
reliability_levels = [0,1,2] #best case, expected, worst case scenario

#to easily test various warehouse capacities if needed for example case
warehouse_cap_level = 3

#define sets
T_set = 12
K_set = 3
I_set = 4

In [4]:
budget_levels

[3000000, 4000000, 5000000]

In [5]:
def initialize_model(reliability_level, 
                     budget_level):
    
    model = ConcreteModel()
    
    #define sets
    T_set = 12
    K_set = 3
    I_set = 4
    
    #supplier-SKU names
    #names_supplier_SKU = ['Supplier 1 - SKU: N95-MG', 'Supplier 1 - SKU: N95-N', 
    # 'Supplier 2 - SKU: SUR-MG', 'Supplier 2 - SKU: SUR-N']

    I_k_dict = {1: [1,2,3,4],
               2: [1,2,3,4],
               3: [1,2,3,4]}

    #####initialize sets#######
    model.K = Set(initialize = range(1,K_set+1))
    model.I = Set(initialize = range(1,I_set+1))
    model.T = Set(initialize = range(1,T_set+1))
    model.T_0 = Set(initialize = range(0,T_set+1)) #to calculate lead time distributions
    model.I_k = Set(model.K)
    for k in model.K:
        for i in range(0,len(I_k_dict[k])):
            model.I_k[k].add(I_k_dict[k][i])
    
    
    ####Healthcare commodity metric parameters
    r_k_dict = {
        1: 1.0,
        2: 0.8,
        3: 0.4
    }

    #suitability score (only specified for suitable items)
    q_k_i_dict = {
        tuple([1,1]):1,
        tuple([1,2]):.9,
        tuple([1,3]):.8,
        tuple([1,4]):.1,
        tuple([2,1]):1,
        tuple([2,2]):.9,
        tuple([2,3]):.8,
        tuple([2,4]):.1,
        tuple([3,1]):1,
        tuple([3,2]):.9,
        tuple([3,3]):.8,
        tuple([3,4]):.1
    }

    def rank_score_initialize(model, k):
        return(r_k_dict.get(k))

    model.r_k = Param(model.K, initialize = rank_score_initialize)

    def suitability_score_initialize(model, k, i):
        if (i in model.I_k[k]):
            return(q_k_i_dict.get(tuple([k,i])))
        else:
            return(None)

    model.q_k_i = Param(model.K, model.I, initialize = suitability_score_initialize)
    
    
    def healthcare_commodity_metric_initialize(model, k, i, t):
        if (i in model.I_k[k]):
            suitability_score = q_k_i_dict.get(tuple([k,i]))
            criticality_score = r_k_dict.get(k)
            phi_k_i_t_temp = suitability_score*(criticality_score*((1+criticality_score)**(T_set-t)))

            return(phi_k_i_t_temp)
        
        else:
            return(None)
        
    model.phi_k_i_t = Param(model.K, model.I, model.T, initialize = healthcare_commodity_metric_initialize)
    

    #######warehouse capacity parameters#######

    #size
    s_i_dict = {
        1: 1,
        2: 1,
        3: 1,
        4: 1
    }

    def item_size_param_initialize(model, i):
        return(s_i_dict.get(i))

    model.s_i = Param(model.I, initialize = item_size_param_initialize)
    
    #total warehouse capacity
    h = 50000*warehouse_cap_level

    model.h = Param(initialize = h)

    #######budget and cost parameters########

    #cost
    c_i_dict = {
        1: 8,
        2: 5,
        3: 10,
        4: 1
    }

    #budget
    b = budget_level

    def cost_param_initialize(model, i):
        return(c_i_dict.get(i))

    model.c_i = Param(model.I, initialize = cost_param_initialize)

    #model.c_k_i.pprint()

    def budget_param_initialize(model):
        return(b)

    model.b = Param(initialize = budget_param_initialize)

    ########inventory availability parameters##########
    #starting inventory
    z_i_init_dict = {
        1 : 5000,
        2 : 5000,
        3 : 5000,
        4 : 5000
    }

    def beg_inv_param_initialize(model, i):
        return(z_i_init_dict.get(i))

    model.z_i_init = Param(model.I, initialize = beg_inv_param_initialize)
    
    #incoming orders
    o_i_t_init_dict = {}


    for i in range(1,I_set+1):
        for t in range(1,T_set+1):
            if t <= 2:
                o_i_t_init_dict[tuple([i,t])] = 2000 
            else:
                o_i_t_init_dict[tuple([i,t])] = 0
                
    def incoming_orders_initialize(model, i, t):
        return(o_i_t_init_dict.get(tuple([i,t])))

    model.o_i_t_init = Param(model.I, model.T, initialize = incoming_orders_initialize)

    #model.o_k_i_t.pprint()

    #######Supply parameters########
    #lead times
    lead_time_dictionary_supplier_SKU = {1: [3,6,4], #min, max, likely, a,b,c
                         2: [2,5,3], 
                         3: [1,3,2],
                         4: [0,0,1]}

    def lead_time_dist(a,b,c):
        lead_time_dist_array = np.zeros(T_set+1)
        cum_prob = 0
        t = 0

        while t < b+1:
            if t+1 <= a:
                t = t+1 
            elif (a < t+1 & t+1 <= c):
                cum_prob_temp = ((t+1-a)**2)/((b+1-a)*(c-a))
                prob = cum_prob_temp - cum_prob
                lead_time_dist_array[t] = prob
                cum_prob = cum_prob_temp
                t = t+1
            elif c < t+1 & t+1 <= b+1:
                cum_prob_temp = 1-((b+1-(t+1))**2)/((b+1-a)*(b+1-c))
                prob = cum_prob_temp - cum_prob
                lead_time_dist_array[t] = prob
                cum_prob = cum_prob_temp
                t = t+1

        return(lead_time_dist_array)

    f_i_l_dict = {}
    for i in model.I:
        lead_time_dist_array = lead_time_dist(lead_time_dictionary_supplier_SKU.get(i)[0],
                                             lead_time_dictionary_supplier_SKU.get(i)[1],
                                             lead_time_dictionary_supplier_SKU.get(i)[2])
        for t in model.T_0:
            f_i_l_dict[tuple([i,t])] = lead_time_dist_array[t]


    #supply capacities
    cap_dictionary_supplier_SKU = {1: np.repeat([42000,44000], [1,T_set-1]), #min, max, likely, a,b,c
                         2: np.repeat([44000,50000], [1,T_set-1]), 
                         3: [44000]*T_set,
                         4: [70000]*T_set}

    cap_dictionary_supplier_SKU_t = {}

    for i in model.I:
        for t in model.T:
            cap_dictionary_supplier_SKU_t[tuple([i,t])] = cap_dictionary_supplier_SKU[i][t-1]

    def delay_dist_param_initialize(model, i, t):
        return(f_i_l_dict.get(tuple([i,t])))

    model.f_i_l = Param(model.I, model.T_0, 
                              initialize = delay_dist_param_initialize)

    def supplier_cap_param_initialize(model, i, t):
        return(cap_dictionary_supplier_SKU_t[i,t])

    model.cap_i_t = Param(model.I, model.T, initialize = supplier_cap_param_initialize)

    ######demand parameters#########

    #unfulfilled demand
    n_k_init_dict = {
        1 : 0,
        2 : 0,
        3 : 0
    }
    

    #projected base incoming demand
    base_demand_k = {1: [7000, 10000, 20000],
                    2: [15000, 17000, 20000],
                    3: [25000, 26000, 27000]}

    #used in this example to adjust demand RV over time
    dynamic_adj_dem_t = {1: [0.5, 0.6, 0.7, 1, 1.1, 1.5, 1.6, 1.2, 1.1, 0.9, 0.8, 0.8],
                        2: [1.5, 1.5, 1.5, 1, 0.8, 0.8, 0.8, 0.8, 1, 1, 1, 1],
                        3: [1,1,1,1,1,1,1,1,1,1,1,1]}
    
    epsilon = reliability_level
    
    def demand_param_initialize(model,k, t):
        base_demand_k_temp = base_demand_k[k]
        adjust_demand_temp = dynamic_adj_dem_t[k][t-1]
        demand_projection_temp = base_demand_k_temp[epsilon]*adjust_demand_temp

        return(demand_projection_temp)
    
    model.d_k_t = Param(model.K, model.T, initialize = demand_param_initialize)

    def unfulfilled_demand_param_initialize(model, k):
        return(n_k_init_dict.get(k))

    model.n_k_init = Param(model.K, initialize = unfulfilled_demand_param_initialize)

    #model.n_k_intialize.pprint()

    ####initialize variables####
    model.x_i_t = Var(model.I, model.T, within = NonNegativeReals) #amount ordered
    model.y_i_t = Var(model.I, model.T, within = NonNegativeReals) #amount recieved
    model.z_i_t = Var(model.I, model.T, within = NonNegativeReals)
    model.n_k_t = Var(model.K, model.T, within = NonNegativeReals)
    model.m_k_i_t = Var(model.K, model.I, model.T, within = NonNegativeReals)
    
    return(model)

In [6]:
def initialize_objective(model):
    model.Objective = Objective(expr = 
                            (sum(model.phi_k_i_t[k,i,t]*model.m_k_i_t[k,i,t] 
                                                      for k in model.K for t in model.T for i in model.I_k[k])),
                            sense = maximize)
    
    return(model)

In [7]:
def initialize_constraints(model):
    
    def initalize_unsatisfied_demand_constraint_initialize(model, k):
        return(model.n_k_t[k,1] == model.n_k_init[k])

    model.initalize_unsatisfied_demand_constraint= \
    Constraint(model.K, rule = initalize_unsatisfied_demand_constraint_initialize)

    def calculate_unsatisfied_demand_constraint_initialize(model,k,t):
        if (t < 12):
            return(sum(model.m_k_i_t[k,i,t] for i in model.I_k[k]) + model.n_k_t[k,t+1] == \
                  model.n_k_t[k,t] + model.d_k_t[k,t])
        else:
            return(sum(model.m_k_i_t[k,i,t] for i in model.I_k[k]) <= model.n_k_t[k,t] + model.d_k_t[k,t])

    model.calculate_unsatisfied_demand_constraint = \
    Constraint(model.K, model.T, rule = calculate_unsatisfied_demand_constraint_initialize)

    def supplier_constraint_initialize(model, i, t):
        return(model.x_i_t[i,t] <= model.cap_i_t[i,t])

    model.supplier_constraint = \
    Constraint(model.I, model.T, rule = supplier_constraint_initialize)

    def incoming_orders_constraint_initialize(model, i, t):
        return(model.y_i_t[i,t] == 
               sum(model.f_i_l[i, t-t_ordered_time]*model.x_i_t[i,t_ordered_time] 
                   for t_ordered_time in range(1,t+1)) + model.o_i_t_init[i,t])

    model.incoming_ordered_constraint = Constraint(model.I, model.T, 
                                                   rule = incoming_orders_constraint_initialize)

    def beggining_inventory_constraint_initialize(model, i):
        return(model.z_i_t[i,1] == model.z_i_init[i])

    model.beggining_inventory_constraint = Constraint(model.I, 
                                                      rule = beggining_inventory_constraint_initialize)

    def calculate_available_inventory_constraint_initialize(model, i, t):
        if (t > 1):
            return(model.z_i_t[i,t] == model.z_i_t[i,t-1] + model.y_i_t[i,t-1] - \
                   sum(model.m_k_i_t[k,i,t-1] for k in model.K))
        else:
            return(Constraint.Skip)

    model.calculate_available_inventory_constraint = Constraint(model.I, model.T,
                                                                rule = 
                                                                calculate_available_inventory_constraint_initialize)

    def cannot_fulfill_more_than_available_constraint_initialize(model, i, t):
        return(sum(model.m_k_i_t[k,i,t] for k in model.K) <= model.z_i_t[i,t])

    model.cannot_fulfill_more_than_available_constraint = \
    Constraint(model.I, model.T, rule = cannot_fulfill_more_than_available_constraint_initialize)

    def warehouse_capacity_constraint_initialize(model, t):
        return(sum(model.s_i[i]*model.z_i_t[i,t] for i in model.I) <= model.h)

    model.warehouse_capacity_constraint = \
    Constraint(model.T, rule = warehouse_capacity_constraint_initialize)

    def budget_constraint_initialize(model):
        return(sum(model.c_i[i]*model.x_i_t[i,t] 
                   for i in model.I for t in model.T) 
               <= model.b)

    model.budget_constraint = \
    Constraint(rule = budget_constraint_initialize)
    
    return(model)

In [8]:
def extract_data(model, reliability_level, budget, 
                 unsatisfied_df, fulfilled_df, ordered_df, demand_df):
    
    #extract from unsatisfied_df
    def unsatisfied_df_update(unsatisfied_df):
        if unsatisfied_df.empty:
            unsatisfied_df = pd.DataFrame(list(model.n_k_t.extract_values().items()),\
                                          columns = ['sets','units_unsatisfied'])
            unsatisfied_df = unsatisfied_df[unsatisfied_df['units_unsatisfied'].notnull()]            
            unsatisfied_df.loc[:,'commodity_type']=unsatisfied_df.sets.map(lambda x:x[0])
            unsatisfied_df.loc[:,'time_interval']=unsatisfied_df.sets.map(lambda x:x[1])
            unsatisfied_df = unsatisfied_df[['commodity_type', 'time_interval', 
                                             'units_unsatisfied']]
            unsatisfied_df ['reliability_level'] = [reliability_level]*len(unsatisfied_df)
            unsatisfied_df['budget'] = [budget]*len(unsatisfied_df)
            return(unsatisfied_df)
        else:
            unsatisfied_df_temp = pd.DataFrame(list(model.n_k_t.extract_values().items()),\
                                          columns = ['sets','units_unsatisfied'])
            unsatisfied_df_temp.loc[:,'commodity_type']=unsatisfied_df_temp.sets.map(lambda x:x[0])
            unsatisfied_df_temp.loc[:,'time_interval']=unsatisfied_df_temp.sets.map(lambda x:x[1])
            unsatisfied_df_temp = unsatisfied_df_temp[['commodity_type', 'time_interval', 
                                                       'units_unsatisfied']]
            unsatisfied_df_temp ['reliability_level'] = [reliability_level]*len(unsatisfied_df_temp)
            unsatisfied_df_temp['budget'] = [budget]*len(unsatisfied_df_temp)
            unsatisfied_df_temp = unsatisfied_df_temp[unsatisfied_df_temp['units_unsatisfied'].notnull()]

            return(unsatisfied_df.append(unsatisfied_df_temp, ignore_index=True))
        
    #extract from satisfied_df
    def fulfilled_df_update(fulfilled_df):
        if fulfilled_df.empty:
            fulfilled_df = pd.DataFrame(list(model.m_k_i_t.extract_values().items()),columns = ['sets','units_fulfilled'])
            fulfilled_df.loc[:,'commodity_type']=fulfilled_df.sets.map(lambda x:x[0])
            fulfilled_df.loc[:,'SKU']=fulfilled_df.sets.map(lambda x:x[1])
            fulfilled_df.loc[:,'time_interval']=fulfilled_df.sets.map(lambda x:x[2])
            fulfilled_df['reliability_level'] = [reliability_level]*len(fulfilled_df)
            fulfilled_df['budget'] = [budget]*len(fulfilled_df)
            fulfilled_df = fulfilled_df[fulfilled_df['units_fulfilled'].notnull()]
            return(fulfilled_df)
        else:
            fulfilled_df_temp = pd.DataFrame(list(model.m_k_i_t.extract_values().items()),columns = ['sets','units_fulfilled'])
            fulfilled_df_temp.loc[:,'commodity_type']=fulfilled_df_temp.sets.map(lambda x:x[0])
            fulfilled_df_temp.loc[:,'SKU']=fulfilled_df_temp.sets.map(lambda x:x[1])
            fulfilled_df_temp.loc[:,'time_interval']=fulfilled_df_temp.sets.map(lambda x:x[2])
            fulfilled_df_temp['reliability_level'] = [reliability_level]*len(fulfilled_df_temp)
            fulfilled_df_temp['budget'] = [budget]*len(fulfilled_df_temp)
            fulfilled_df_temp = fulfilled_df_temp[fulfilled_df_temp['units_fulfilled'].notnull()]

            return(fulfilled_df.append(fulfilled_df_temp, ignore_index=True))
        
    def ordered_df_update(ordered_df):
        if ordered_df.empty:
            ordered_df = pd.DataFrame(list(model.x_i_t.extract_values().items()),columns = ['sets','units_to_order'])
            ordered_df.loc[:,'SKU']=ordered_df.sets.map(lambda x:x[0])
            ordered_df.loc[:,'time_interval']=ordered_df.sets.map(lambda x:x[1])
            ordered_df['reliability_level'] = [reliability_level]*len(ordered_df)
            ordered_df['budget'] = [budget]*len(ordered_df)
            
            return(ordered_df)
        else:
            ordered_df_temp = pd.DataFrame(list(model.x_i_t.extract_values().items()),columns = ['sets','units_to_order'])
            ordered_df_temp.loc[:,'SKU']=ordered_df_temp.sets.map(lambda x:x[0])
            ordered_df_temp.loc[:,'time_interval']=ordered_df_temp.sets.map(lambda x:x[1])
            ordered_df_temp['reliability_level'] = [reliability_level]*len(ordered_df_temp)
            ordered_df_temp['budget'] = [budget]*len(ordered_df_temp)
            
            return(ordered_df.append(ordered_df_temp, ignore_index = True))
    
    #extract from unsatisfied_df
    def demand_df_update(demand_df):
        if demand_df.empty:
            demand_df = pd.DataFrame(list(model.d_k_t.extract_values().items()),\
                                          columns = ['sets','incoming_demand'])
            demand_df = demand_df[demand_df['incoming_demand'].notnull()]            
            demand_df.loc[:,'commodity_type']=demand_df.sets.map(lambda x:x[0])
            demand_df.loc[:,'time_interval']=demand_df.sets.map(lambda x:x[1])
            demand_df = demand_df[['commodity_type', 'time_interval', 
                                             'incoming_demand']]
            demand_df['reliability_level'] = [reliability_level]*len(demand_df)
            return(demand_df)
        else:
            demand_df_temp = pd.DataFrame(list(model.d_k_t.extract_values().items()),\
                                          columns = ['sets','incoming_demand'])
            demand_df_temp.loc[:,'commodity_type']=demand_df_temp.sets.map(lambda x:x[0])
            demand_df_temp.loc[:,'time_interval']=demand_df_temp.sets.map(lambda x:x[1])
            demand_df_temp = demand_df_temp[['commodity_type', 'time_interval', 
                                                       'incoming_demand']]
            demand_df_temp ['reliability_level'] = [reliability_level]*len(demand_df_temp)

            return(demand_df.append(demand_df_temp, ignore_index=True))
    
    unsatisfied_df = unsatisfied_df_update(unsatisfied_df)
    fulfilled_df = fulfilled_df_update(fulfilled_df)
    ordered_df = ordered_df_update(ordered_df)
    demand_df = demand_df_update(demand_df)
    
    return(unsatisfied_df, fulfilled_df, ordered_df, demand_df)

In [9]:
fulfilled_df_plan = pd.DataFrame()
unsatisfied_df_plan = pd.DataFrame()
ordered_df_plan = pd.DataFrame()
demand_df_plan = pd.DataFrame()

for r in reliability_levels:
    for b in budget_levels:
        model = initialize_model(r, b)
        model = initialize_objective(model)
        model = initialize_constraints(model)
        opt = pyomo.opt.SolverFactory("glpk")
        results = opt.solve(model)
        opt.solve(model)

        unsatisfied_df_plan, fulfilled_df_plan, ordered_df_plan, demand_df_plan = \
        extract_data(model, r, b, unsatisfied_df_plan, 
                     fulfilled_df_plan, ordered_df_plan, demand_df_plan)
        
        
demand_df_plan = \
demand_df_plan.drop_duplicates(subset=['reliability_level',
                                       'commodity_type',
                                      'time_interval'])

In [10]:
unsatisfied_df_plan[(unsatisfied_df_plan['reliability_level'] == 1)\
                   &(unsatisfied_df_plan['commodity_type'] == 2)].tail(12)

,commodity_type,time_interval,units_unsatisfied,reliability_level,budget
192,2,1,0.000000,1,5000000
193,2,2,10500.000000,1,5000000
194,2,3,36000.000000,1,5000000
195,2,4,47833.333333,1,5000000
196,2,5,27166.666667,1,5000000
197,2,6,0.000000,1,5000000
198,2,7,0.000000,1,5000000
199,2,8,0.000000,1,5000000
200,2,9,0.000000,1,5000000
201,2,10,0.000000,1,5000000


In [11]:
#set up delay df to calculate average delay
#group filfilled and unsatisfied demand over item type (over all supplier-SKUs)
delay_ref_df = fulfilled_df_plan.groupby(['commodity_type', 'time_interval', 
                      'reliability_level', 'budget'])['units_fulfilled'].sum().reset_index()

#create a unique id to calculate cum fulfilled (for calculating delay) based on reliability level, budget and item type
delay_ref_df['unique_id'] = delay_ref_df.groupby(['reliability_level', 'budget', 'commodity_type']).ngroup()
delay_ref_df = delay_ref_df.sort_values(by=['unique_id', 'time_interval', 'commodity_type'])
delay_ref_df = delay_ref_df.reset_index(drop = True)

unique_id_itr = 0
cum_units_fulfilled_array = np.zeros(len(delay_ref_df))

for row in range(len(delay_ref_df)):
    units_fulfilled_temp = delay_ref_df['units_fulfilled'][row]
    
    if delay_ref_df['unique_id'][row] == unique_id_itr:
        cum_units_fulfilled_array[row] = units_fulfilled_temp+cum_units_fulfilled_array[row-1]
    else:
        cum_units_fulfilled_array[row] = units_fulfilled_temp 
        
    unique_id_itr = delay_ref_df['unique_id'][row]

In [12]:
delay_ref_df = pd.merge(delay_ref_df, demand_df_plan,
                    how = 'left', on = ['commodity_type','time_interval',
                                        'reliability_level'])

commodity_relibaility_ref_id_df = delay_ref_df.drop_duplicates(subset=['unique_id'])

commodity_relibaility_ref_id_df = commodity_relibaility_ref_id_df[['unique_id',
                                                                  'commodity_type',
                                                                  'reliability_level',
                                                                  'budget']]

In [13]:
commodity_relibaility_ref_id_df

,unique_id,commodity_type,reliability_level,budget
0,0,1,0,3000000
12,1,2,0,3000000
24,2,3,0,3000000
36,3,1,0,4000000
48,4,2,0,4000000
60,5,3,0,4000000
72,6,1,0,5000000
84,7,2,0,5000000
96,8,3,0,5000000
108,9,1,1,3000000


In [14]:
demand_fulfillment_schedule_df = pd.DataFrame(columns=['unique_id', 'time_interval', 
                                                      'units_fulfilled', 'incoming_demand'])

for newcol in range(1,T_set+1):
    demand_fulfillment_schedule_df[newcol] = ""
    
for unique_id_itr in delay_ref_df['unique_id'].unique():
    delay_ref_df_temp = delay_ref_df[delay_ref_df['unique_id'] == unique_id_itr]
    fulfilled_remaining = (delay_ref_df_temp['units_fulfilled']).to_numpy()
    incoming_demand_remaining = (delay_ref_df_temp['incoming_demand']).to_numpy()
    
    for t1 in range(1,T_set+1):
        demand_fulfillment_schedule_df = demand_fulfillment_schedule_df.append({'unique_id': unique_id_itr,
                                                                                'time_interval': t1,
                                                                                'units_fulfilled': fulfilled_remaining[t1-1],
                                                                                'incoming_demand': incoming_demand_remaining[t1-1]},
                                                                               #'remaining_demand': incoming_demand_remaining[t1-1],
                                                                               #'remaining_units_fulfilled': fulfilled_remaining[t1-1]},
                                                                               ignore_index=True)  
        
demand_fulfillment_schedule_df = demand_fulfillment_schedule_df.fillna(0)

In [15]:
for unique_id_itr in delay_ref_df['unique_id'].unique():
    delay_ref_df_temp = delay_ref_df[delay_ref_df['unique_id'] == unique_id_itr]
    fulfilled_orig_temp = (delay_ref_df_temp['units_fulfilled']).to_numpy()
    demand_orig_temp = (delay_ref_df_temp['incoming_demand']).to_numpy()
    
    time_requested = 1
    time_fulfilled = 1
    
    while time_requested <= T_set and time_fulfilled <= T_set:
        
        #calculate the amount of the fulfilled demand accounted for at time_fulfilled
        demand_fulfillment_schedule_df_temp = demand_fulfillment_schedule_df[demand_fulfillment_schedule_df['unique_id']==unique_id_itr]
        amt_fulfilled_accounted_for = sum(demand_fulfillment_schedule_df_temp[time_fulfilled])
        
        fulfilled_remaining = fulfilled_orig_temp[time_fulfilled-1]-amt_fulfilled_accounted_for
        
        #calculate the amount of demand accounted for at time_requested
        demand_index = demand_fulfillment_schedule_df[(demand_fulfillment_schedule_df['time_interval'] == time_requested) &
                                      (demand_fulfillment_schedule_df['unique_id'] == unique_id_itr)].index[0]
        amt_demand_fulfilled = sum(demand_fulfillment_schedule_df.iloc[demand_index,-T_set:])
        
        demand_remaining = demand_orig_temp[time_requested-1]-amt_demand_fulfilled
        
        
        #the amount fulfilled at during time interval time_fulfilled, 
        #that was orig. requested at time time_requested is the min of fulfilled and demand remaining
        
        amt_fulfilled = min(fulfilled_remaining, demand_remaining)
        
        demand_fulfillment_schedule_df.loc[(demand_fulfillment_schedule_df['time_interval'] == time_requested) \
                                   & (demand_fulfillment_schedule_df['unique_id'] == unique_id_itr),\
                                   time_fulfilled] = amt_fulfilled
        
            
            
        #if there is no more remaining demand requests unassigned orig. requested at time time_requested
        #AND there is no more unassigned fulfilled demand from time time_fulfilled
        if (fulfilled_remaining == demand_remaining):
            time_requested = time_requested+1
            time_fulfilled = time_fulfilled+1
            
        #if there is remaining demand requests, but no more fulfilled demand
        if (fulfilled_remaining < demand_remaining):
            time_fulfilled = time_fulfilled+1
            
        #if there is remaining fulfilled demand, but no more demand requests
            
        if (fulfilled_remaining > demand_remaining):
            time_requested = time_requested+1
        

demand_fulfillment_schedule_df['units_UNfulfilled'] = demand_fulfillment_schedule_df['incoming_demand']-\
demand_fulfillment_schedule_df.loc[:,demand_fulfillment_schedule_df.columns[-T_set:]].sum(1)

In [16]:
demand_fulfillment_schedule_df

,unique_id,time_interval,units_fulfilled,incoming_demand,1,2,3,4,5,6,7,8,9,10,11,12,units_UNfulfilled
0,0.0,1.0,3500.0,3500.0,3500.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0
1,0.0,2.0,4200.0,4200.0,0.0,4200.0,9.094947e-13,0.000000e+00,0.0,0.0,0.0,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0
2,0.0,3.0,4900.0,4900.0,0.0,0.0,4.900000e+03,0.000000e+00,0.0,0.0,0.0,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0
3,0.0,4.0,7000.0,7000.0,0.0,0.0,6.366463e-12,7.000000e+03,0.0,0.0,0.0,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0
4,0.0,5.0,7700.0,7700.0,0.0,0.0,0.000000e+00,3.637979e-11,7700.0,0.0,0.0,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
319,26.0,8.0,27000.0,27000.0,0.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0.0,27000.0,1.018634e-10,0.000000e+00,0.000000e+00,0.000000e+00,0.0
320,26.0,9.0,27000.0,27000.0,0.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.0,2.700000e+04,4.001777e-10,0.000000e+00,0.000000e+00,0.0
321,26.0,10.0,27000.0,27000.0,0.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.0,0.000000e+00,2.700000e+04,4.001777e-10,0.000000e+00,0.0
322,26.0,11.0,27000.0,27000.0,0.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.0,0.000000e+00,0.000000e+00,2.700000e+04,4.401954e-10,0.0


In [17]:
demand_fulfillment_schedule_df.to_csv('demand_fulfillment_schedule_df.csv', index=False)

In [18]:
demand_fulfillment_schedule_df.head()

,unique_id,time_interval,units_fulfilled,incoming_demand,1,2,3,4,5,6,7,8,9,10,11,12,units_UNfulfilled
0,0.0,1.0,3500.0,3500.0,3500.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,2.0,4200.0,4200.0,0.0,4200.0,9.094947e-13,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,3.0,4900.0,4900.0,0.0,0.0,4.900000e+03,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,4.0,7000.0,7000.0,0.0,0.0,6.366463e-12,7.000000e+03,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,5.0,7700.0,7700.0,0.0,0.0,0.000000e+00,3.637979e-11,7700.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [19]:
os.getcwd()

'/Users/chelseagreene/github/epi_supplychain_optimization/inventory_management/model/publication/journal_of_humanitarian_log_and_scm/outputs/strategic_decisions'

In [20]:
#calculate average delay from delay fulfillment schedule
delay_calcs_df=pd.melt(demand_fulfillment_schedule_df, 
                       id_vars=['unique_id', 'time_interval',
                                'units_fulfilled', 'incoming_demand', 'units_UNfulfilled'])
delay_calcs_df = delay_calcs_df.rename(columns={"variable": "time_interval_fulfilled", 
                               "time_interval": "time_interval_requested"})

delay_calcs_df=delay_calcs_df[['unique_id', 'time_interval_fulfilled', 'time_interval_requested', 'value']]
delay_calcs_df = delay_calcs_df[delay_calcs_df['value']>0]
delay_calcs_df['delay'] = delay_calcs_df['time_interval_fulfilled']-delay_calcs_df['time_interval_requested']
delay_calcs_df.loc[delay_calcs_df['delay']>0, 'is_delayed'] = 1
delay_calcs_df.loc[delay_calcs_df['delay']==0, 'is_delayed'] = 0
delay_calcs_df.loc[delay_calcs_df['delay']>1, 'is_delayed_2'] = 1
delay_calcs_df.loc[delay_calcs_df['delay']<=1, 'is_delayed_2'] = 0
delay_calcs_df.loc[delay_calcs_df['delay']>2, 'is_delayed_3'] = 1
delay_calcs_df.loc[delay_calcs_df['delay']<=2, 'is_delayed_3'] = 0

In [30]:
delay_calcs_df['avg_delay'] = delay_calcs_df['value']*delay_calcs_df['delay']

In [31]:
#### to do calculate average delay for each unique id.

delay_calcs_df.groupby(by = [''])

,unique_id,time_interval_fulfilled,time_interval_requested,value,delay,is_delayed,is_delayed_2,is_delayed_3,avg_delay
0,0.000,1,1.000,3500.000,0.000,0.000,0.000,0.000,0.000
12,1.000,1,1.000,16500.000,0.000,0.000,0.000,0.000,0.000
36,3.000,1,1.000,3500.000,0.000,0.000,0.000,0.000,0.000
48,4.000,1,1.000,16500.000,0.000,0.000,0.000,0.000,0.000
72,6.000,1,1.000,3500.000,0.000,0.000,0.000,0.000,0.000
...,...,...,...,...,...,...,...,...,...
3862,24.000,12,11.000,0.000,1.000,1.000,0.000,0.000,0.000
3863,24.000,12,12.000,16000.000,0.000,0.000,0.000,0.000,0.000
3875,25.000,12,12.000,20000.000,0.000,0.000,0.000,0.000,0.000
3886,26.000,12,11.000,0.000,1.000,1.000,0.000,0.000,0.000


In [21]:
delay_calcs_df1 = delay_calcs_df.groupby(by = ['unique_id', 'is_delayed'], as_index=False).agg({"value": "sum"})
delay_calcs_df1['amount_delayed'] = delay_calcs_df1['value']*delay_calcs_df1['is_delayed']
delay_calcs_df1 = delay_calcs_df1.groupby(by = ['unique_id'], as_index=False).agg({"value": "sum",
                                                                                  "amount_delayed": "sum"})
delay_calcs_df1['percent_delayed'] = delay_calcs_df1['amount_delayed']/delay_calcs_df1['value']
delay_calcs_df1 = delay_calcs_df1[['unique_id', 'percent_delayed']]


delay_calcs_df2 = delay_calcs_df.groupby(by = ['unique_id', 'is_delayed_2'], as_index=False).agg({"value": "sum"})
delay_calcs_df2['amount_delayed'] = delay_calcs_df2['value']*delay_calcs_df2['is_delayed_2']
delay_calcs_df2 = delay_calcs_df2.groupby(by = ['unique_id'], as_index=False).agg({"value": "sum",
                                                                                  "amount_delayed": "sum"})
delay_calcs_df2['percent_delayed_by_at_least_2'] = delay_calcs_df2['amount_delayed']/delay_calcs_df2['value']
delay_calcs_df2 = delay_calcs_df2[['unique_id', 'percent_delayed_by_at_least_2']]


delay_calcs_df3 = delay_calcs_df.groupby(by = ['unique_id', 'is_delayed_3'], as_index=False).agg({"value": "sum"})
delay_calcs_df3['amount_delayed'] = delay_calcs_df3['value']*delay_calcs_df3['is_delayed_3']
delay_calcs_df3 = delay_calcs_df3.groupby(by = ['unique_id'], as_index=False).agg({"value": "sum",
                                                                                  "amount_delayed": "sum"})
delay_calcs_df3['percent_delayed_by_at_least_3'] = delay_calcs_df3['amount_delayed']/delay_calcs_df3['value']
delay_calcs_df3 = delay_calcs_df3[['unique_id', 'percent_delayed_by_at_least_3']]

In [22]:
delay_calcs_df_metric = delay_calcs_df1.merge(delay_calcs_df2, on='unique_id', how='left')
delay_calcs_df_metric = delay_calcs_df_metric.merge(delay_calcs_df3, on='unique_id', how='left')
delay_calcs_df_metric = delay_calcs_df_metric.merge(commodity_relibaility_ref_id_df,
                          on = 'unique_id', how = 'left')

In [23]:
delay_calcs_df_metric.head()

,unique_id,percent_delayed,percent_delayed_by_at_least_2,percent_delayed_by_at_least_3,commodity_type,reliability_level,budget
0,0.0,7.707582e-17,0.000000,0.000000,1,0,3000000
1,1.0,3.310798e-01,0.085258,0.000000,2,0,3000000
2,2.0,6.666667e-01,0.409344,0.254025,3,0,3000000
3,3.0,9.249099e-16,0.000000,0.000000,1,0,4000000
4,4.0,3.084864e-01,0.079440,0.000000,2,0,4000000


In [24]:
#change from scientific to float
pd.set_option('display.float_format', lambda x: '%0.3f' % x)

In [25]:
delay_calcs_df_metric

,unique_id,percent_delayed,percent_delayed_by_at_least_2,percent_delayed_by_at_least_3,commodity_type,reliability_level,budget
0,0.000,0.000,0.000,0.000,1,0,3000000
1,1.000,0.331,0.085,0.000,2,0,3000000
2,2.000,0.667,0.409,0.254,3,0,3000000
3,3.000,0.000,0.000,0.000,1,0,4000000
4,4.000,0.308,0.079,0.000,2,0,4000000
5,5.000,0.458,0.333,0.224,3,0,4000000
6,6.000,0.000,0.000,0.000,1,0,5000000
7,7.000,0.308,0.079,0.000,2,0,5000000
8,8.000,0.417,0.333,0.224,3,0,5000000
9,9.000,0.000,0.000,0.000,1,1,3000000


In [ ]:
# Build the plot
fig, ax = plot.subplots()
ax.bar(x_pos, CTEs, yerr=error, align='center', alpha=0.5, ecolor='black', capsize=10)
ax.set_ylabel('Coefficient of Thermal Expansion ($\degree C^{-1}$)')
ax.set_xticks(x_pos)
ax.set_xticklabels(materials)
ax.set_title('Coefficent of Thermal Expansion (CTE) of Three Metals')
ax.yaxis.grid(True)

# Save the figure and show
plt.tight_layout()
plt.savefig('bar_plot_with_error_bars.png')
plt.show()

In [ ]:
#average quality of fulfilled items

q_k_i_dict = {
    tuple([1,1]):1,
    tuple([1,2]):.9,
    tuple([1,3]):.8,
    tuple([1,4]):.7
}

quality_array = np.empty(len(fulfilled_df_plan))

for i in fulfilled_df_plan.index:
    it = fulfilled_df_plan['item_type'][i]
    ss = fulfilled_df_plan['supplier'][i]
    quality_array[i] = q_k_i_dict.get(tuple([it, ss]))
    
fulfilled_df_plan['quality'] = quality_array

fulfilled_df_quality = \
fulfilled_df_plan[fulfilled_df_plan['units_fulfilled']>0]

fulfilled_df_quality['weighted_quality'] = fulfilled_df_quality['quality']*\
fulfilled_df_quality['units_fulfilled']

fulfilled_df_quality_metric = fulfilled_df_quality\
.groupby(by = ['reliability_level', 'budget'], as_index = False).\
agg({"weighted_quality": "sum",
    "units_fulfilled": "sum"})

fulfilled_df_quality_metric['average_quality'] = \
fulfilled_df_quality_metric['weighted_quality']/\
fulfilled_df_quality_metric['units_fulfilled']

In [ ]:
fulfilled_df_quality_metric

In [ ]:
delay_ref_df

In [ ]:
quality_delay_metric = delay_ref_df[['unique_id', 'reliability_level', 'budget']].\
drop_duplicates(subset=['unique_id'])

quality_delay_metric = quality_delay_metric[['unique_id', 'reliability_level', 'budget']]\
.merge(fulfilled_df_quality_metric[['reliability_level', 'budget', 'average_quality']], 
       on=['reliability_level', 'budget'], how='left')

quality_delay_metric = quality_delay_metric.merge(delay_calcs_df_metric, on = ['unique_id'], how = 'left')

In [ ]:
quality_delay_metric

In [ ]:
fulfilled_df_item_time = fulfilled_df_plan.groupby(['item_type', 'time_interval', 
                      'reliability_level', 'budget'])['units_fulfilled'].sum().reset_index()

unsatisfied_df_item_time = unsatisfied_df_plan.groupby(['item_type', 'time_interval', 
                      'reliability_level', 'budget'])['units_unsatisfied'].sum().reset_index()

reliability_df = pd.merge(fulfilled_df_item_time[['item_type', 'time_interval', 'units_fulfilled',
                                                  'reliability_level', 'budget']], 
                          unsatisfied_df_item_time[['item_type', 'time_interval', 'units_unsatisfied',
                                          'reliability_level', 'budget']], 
                          how='left', on=['item_type','time_interval',
                                         'reliability_level', 'budget'])

reliability_df['total_number_of_open_requests'] = reliability_df['units_fulfilled'] +\
reliability_df['units_unsatisfied'] 

reliability_df['percent_unsatisfied'] = \
reliability_df['units_unsatisfied']/reliability_df['total_number_of_open_requests']

reliability_df_grouped = \
reliability_df.groupby(['reliability_level', 'budget'])['percent_unsatisfied'].sum().reset_index()

reliability_df = pd.merge(fulfilled_df_item_time[['item_type', 'time_interval', 'units_fulfilled',
                                                  'reliability_level', 'budget']], 
                          unsatisfied_df_plan[['item_type', 'time_interval', 'units_unsatisfied',
                                          'reliability_level', 'budget']], 
                          how='left', on=['item_type','time_interval',
                                         'reliability_level', 'budget'])

reliability_df_grouped['risk_of_understock'] = (reliability_df_grouped['percent_unsatisfied']/(T_set*K_set))

reliability_df_grouped['budget_thousands'] = reliability_df_grouped['budget']/1000
reliability_df_grouped = reliability_df_grouped[reliability_df_grouped['budget_thousands'] <= max(unsatisfied_df_plan['budget'])]
reliability_df_grouped['percent_unsatisfied'] = (reliability_df_grouped['risk_of_understock']*100).astype(int)


#set lengend title size of plots
plot.rcParams['legend.title_fontsize'] = 'Large'

In [ ]:
reliability_df_grouped

In [ ]:
fig, ax = plot.subplots(figsize=(6,6))


ax.plot('budget_thousands', 'percent_unsatisfied', 
          data=reliability_df_grouped[reliability_df_grouped['reliability_level'] == reliability_levels[0]],
         marker='X', color = 'black', label = '50%', markersize=8)#r'$\theta^D = \theta^{CAP}$ = 0' + ' (Assuming Expected Demand and Supplier Capacity)')
ax.plot('budget_thousands', 'percent_unsatisfied', 
          data=reliability_df_grouped[reliability_df_grouped['reliability_level'] == reliability_levels[1]],
         marker='o', color = 'black', label = '75%', markersize=8, mfc = 'white') #r'$\theta^D = \theta^{CAP}$ = 1' + ' (Assuming Some Variability in Demand and Supplier Capacity)')
ax.plot('budget_thousands', 'percent_unsatisfied', 
          data=reliability_df_grouped[reliability_df_grouped['reliability_level'] == reliability_levels[2]],
         marker='D', color = 'gray', label = '95%', markersize = 8)#r'$\theta^D = \theta^{CAP}$ = 2' + ' (high reliability)')

ax.set_xlabel('Budget (in thousands)', fontsize = 13)
ax.set_ylabel('Overall Percent of Unsatisfied Demand', fontsize = 13)

#ax.set(ylim=(0, 1), xlim=(0, max(reliability_df_grouped['budget'])))
ax.set(ylim=(0, 100), xlim=(0, 2500))
#ax.set_title(r'$\bf{Overall ~ Expected ~Percent ~of ~Unsatisfied ~Demand}$'+
#             '\n'+ r'$\bf{Under ~Various~ Budgets ~and ~Variability~ in~Demand ~and ~Supplier~Capacity~Assumptions~}$' +
#             '\n \n \n where the expected percent of unsatisfied demand for each item during week'#+ r'$(t)$' + ', represented by: \n'+\
              #r'$\psi_{k,t} = \frac{\alpha_{k,t}}{\alpha_{k,t} + \sum_{i \in I} \beta_{k,i,t}} \forall k \in K, t \in T$' +
#             '\n ...is averaged over all items and time intervals to obtain the '#+  r'$(K)$'+ ' and time intervals '+  r'$(T)$'+ ' to obtain the:'\
#             '\n overall expected percent of unsatisfied demand',# + r'$\frac{\sum_{t \in T} \sum_{k \in K} \psi_{k,t}}{K \times T}$',
#             fontsize = 16)

ax.yaxis.set_major_formatter(mtick.PercentFormatter())

plot.legend(title = 'Demand and Supply \n Capacity Reliability', 
            fontsize=13, 
            fancybox=True)._legend_box.align='center'

plot.grid(color='gray', linestyle='-', linewidth=.4, axis = 'y')


#os.chdir(os.getcwd()+'/outputs/3_2_1_strategic_decisions')
#plot.savefig('budget_high_WH.jpg')
plot.savefig('budget_low_WH.jpg')